# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff04cc1c070>
├── 'a' --> tensor([[ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170]])
└── 'x' --> <FastTreeValue 0x7ff04cc5da00>
    └── 'c' --> tensor([[-0.5979,  0.0751,  0.4851,  0.0678],
                        [-1.0033, -1.4178, -0.3906, -0.9145],
                        [-1.1393,  0.5521,  0.2966,  0.5511]])

In [4]:
t.a

tensor([[ 0.8201,  0.3666,  0.8445],
        [-0.5510, -1.1752, -0.8170]])

In [5]:
%timeit t.a

63.8 ns ± 0.268 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff04cc1c070>
├── 'a' --> tensor([[ 2.9355, -0.6397,  0.6268],
│                   [-1.2321, -1.1942,  1.5732]])
└── 'x' --> <FastTreeValue 0x7ff04cc5da00>
    └── 'c' --> tensor([[-0.5979,  0.0751,  0.4851,  0.0678],
                        [-1.0033, -1.4178, -0.3906, -0.9145],
                        [-1.1393,  0.5521,  0.2966,  0.5511]])

In [7]:
%timeit t.a = new_value

63.5 ns ± 0.0668 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170]]),
    x: Batch(
           c: tensor([[-0.5979,  0.0751,  0.4851,  0.0678],
                      [-1.0033, -1.4178, -0.3906, -0.9145],
                      [-1.1393,  0.5521,  0.2966,  0.5511]]),
       ),
)

In [10]:
b.a

tensor([[ 0.8201,  0.3666,  0.8445],
        [-0.5510, -1.1752, -0.8170]])

In [11]:
%timeit b.a

54.2 ns ± 0.053 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.1359,  0.8283, -0.5382],
               [-0.9880, -0.7601,  1.8791]]),
    x: Batch(
           c: tensor([[-0.5979,  0.0751,  0.4851,  0.0678],
                      [-1.0033, -1.4178, -0.3906, -0.9145],
                      [-1.1393,  0.5521,  0.2966,  0.5511]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.123 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 13.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 51.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 702 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fef6335ec10>
├── 'a' --> tensor([[[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]],
│           
│                   [[ 0.8201,  0.3666,  0.8445],
│                    [-0.5510, -1.1752, -0.8170]]])
└── 'x' --> <FastTreeValue 0x7ff04cc5d970>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 76.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fef6335ed30>
├── 'a' --> tensor([[ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170],
│                   [ 0.8201,  0.3666,  0.8445],
│                   [-0.5510, -1.1752, -0.8170]])
└── 'x' --> <FastTreeValue 0x7ff04cc632e0>
    └── 'c' --> tensor([[-0.5979,  0.0751,  0.4851,  0.0678],
                        [-1.0033, -1.4178, -0.3906, -0.9145],
                 

In [23]:
%timeit t_cat(trees)

31.1 µs ± 27.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.4 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]],
       
               [[ 0.8201,  0.3666,  0.8445],
                [-0.5510, -1.1752, -0.8170]]]),
    x: Batch(
           c: tensor([[[-0.5979,  0.0751,  0.4851,  0.0678],
                       [-1.0033, -1.4178, -0.3906, -0.9145],
                       [-1.1393,  0.5521,  0.2966,  0.5511]],
         

In [26]:
%timeit Batch.stack(batches)

79.9 µs ± 185 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170],
               [ 0.8201,  0.3666,  0.8445],
               [-0.5510, -1.1752, -0.8170]]),
    x: Batch(
           c: tensor([[-0.5979,  0.0751,  0.4851,  0.0678],
                      [-1.0033, -1.4178, -0.3906, -0.9145],
                      [-1.1393,  0.5521,  0.2966,  0.5511],
                      [-0.5979,  0.0751,  0.4851,  0.0678],
                      [-1.0033, -

In [28]:
%timeit Batch.cat(batches)

150 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

358 µs ± 4.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
